In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import os
from numpy import linalg as LA

In [2]:
img = Image.open('./Picture/cat-1.jpg')
img = img.resize((img.size[0],img.size[0]*2 ), Image.LANCZOS)
img_pixel = np.array(img)

In [3]:
height, width, channel = img_pixel.shape

In [4]:
error_id = []
def check(input1, input2):
    global error_id
    error_id = []
    sum = 0
    for i in range(len(input1)):
        for j in range(len(input1[0])):
            if np.abs(input1[i][j]-input2[i][j])!=0:
                error_id.append([i,j])
            sum+=np.abs(input1[i][j]-input2[i][j])
    return sum

In [5]:
def check_avg(input1, input2):
    sum = 0
    count = 0
    for i in range(len(input1)):
        for j in range(len(input1[0])):
            sum+=np.abs(input1[i][j]-input2[i][j])
            count+=1
    return sum/count

In [6]:
def print_array(arr):
    for r in range(len(arr)):
        for c in range(len(arr[0])):
            print(arr[r][c], end='\t')
        print()

In [7]:
def gray_scale(input, channel):
    if channel==1:
        return
    return 0.299*input[:,:,0]+0.587*input[:,:,1]+0.114*input[:,:,2]

In [8]:
gray = gray_scale(img_pixel, channel)
gray = [[np.rint(e) for e in row] for row in gray]

In [9]:
filter_x = [[-1, 0, 1]]
filterx_size = (1,3)
filter_y = [[-1],
            [0],
            [1]]
filtery_size = (3, 1)

In [10]:
def convolution(input, width, height, filter, filter_size):
    output = []
    for h in range(height):
        row = []
        for w in range(width):
            current_sum = 0
            for f_r in range(filter_size[0]):
                for f_c in range(filter_size[1]):
                    pixel_r = h - int(filter_size[0]/2) + f_r
                    pixel_c = w - int(filter_size[1]/2) + f_c
                    pixel_r = min(max(0, pixel_r), height - 1)
                    pixel_c = min(max(0, pixel_c), width - 1)
                    current_sum += filter[f_r][f_c] * input[pixel_r][pixel_c]
            row.append(current_sum)
        output.append(row)
    return output

In [11]:
gradient_x = convolution(gray, width, height, filter_x, filterx_size)
gradient_y = convolution(gray, width, height, filter_y, filtery_size)

In [12]:
gradient_x = np.array(gradient_x)
gradient_y = np.array(gradient_y)

In [13]:
gradient_x = gradient_x.astype('float32')
gradient_y = gradient_y.astype('float32')

In [14]:
my_magnitude = np.sqrt(np.square(gradient_x) + np.square(gradient_y))

In [15]:
my_magnitude

array([[ 1.       ,  1.       ,  1.       , ...,  2.236068 ,  4.1231055,
         3.       ],
       [ 1.       ,  1.       ,  1.       , ...,  3.1622777,  3.1622777,
         2.       ],
       [ 0.       ,  0.       ,  0.       , ...,  2.       ,  3.       ,
         2.       ],
       ...,
       [ 6.0827627, 12.206555 , 10.770329 , ...,  2.       ,  5.0990195,
         4.1231055],
       [ 9.433981 , 15.652476 ,  9.486833 , ...,  2.       ,  5.       ,
         4.       ],
       [ 8.5440035, 14.3178215,  5.       , ...,  2.       ,  5.       ,
         4.       ]], dtype=float32)

In [16]:
magnitude

NameError: name 'magnitude' is not defined

In [17]:
my_direction = np.rad2deg(np.arctan(np.array(gradient_y), np.array(gradient_x))) % 180

In [18]:
t2 = np.rad2deg(np.arctan2(gradient_y,gradient_x)) % 180

In [19]:
t2

array([[ 90.      ,  90.      ,  90.      , ...,  26.565048,  14.036255,
          0.      ],
       [ 90.      ,  90.      ,  90.      , ...,  18.434952,  18.434952,
          0.      ],
       [  0.      ,   0.      ,   0.      , ...,   0.      ,   0.      ,
          0.      ],
       ...,
       [  9.462326,  34.992035,  68.1986  , ...,   0.      , 168.69006 ,
         14.036255],
       [ 32.0054  ,  26.565048,  71.565056, ...,   0.      ,   0.      ,
          0.      ],
       [ 20.55606 ,  12.094772,  36.869904, ...,   0.      ,   0.      ,
          0.      ]], dtype=float32)

In [70]:
check_avg(t2, t)

89.99994752544828

In [41]:
orientation

array([[5.79833984e-04, 5.79833984e-04, 5.79833984e-04, ...,
        1.16565163e+02, 1.04036278e+02, 9.00000000e+01],
       [5.79833984e-04, 5.79833984e-04, 5.79833984e-04, ...,
        1.08435005e+02, 1.08435005e+02, 9.00000000e+01],
       [9.00000000e+01, 9.00000000e+01, 9.00000000e+01, ...,
        9.00000000e+01, 9.00000000e+01, 9.00000000e+01],
       ...,
       [9.94623337e+01, 1.24992043e+02, 1.58198639e+02, ...,
        9.00000000e+01, 7.86900482e+01, 1.04036278e+02],
       [1.22005417e+02, 1.16565063e+02, 1.61565109e+02, ...,
        9.00000000e+01, 9.00000000e+01, 9.00000000e+01],
       [1.10556068e+02, 1.02094765e+02, 1.26869965e+02, ...,
        9.00000000e+01, 9.00000000e+01, 9.00000000e+01]], dtype=float32)

In [69]:
t = np.arctan(np.divide(dy, dx+0.00001)) # radian
t = np.degrees(t) # -90 -> 90
t += 90 # 0 -> 180
t

array([[5.79833984e-04, 5.79833984e-04, 5.79833984e-04, ...,
        1.16565163e+02, 1.04036278e+02, 9.00000000e+01],
       [5.79833984e-04, 5.79833984e-04, 5.79833984e-04, ...,
        1.08435005e+02, 1.08435005e+02, 9.00000000e+01],
       [9.00000000e+01, 9.00000000e+01, 9.00000000e+01, ...,
        9.00000000e+01, 9.00000000e+01, 9.00000000e+01],
       ...,
       [9.94623337e+01, 1.24992043e+02, 1.58198639e+02, ...,
        9.00000000e+01, 7.86900482e+01, 1.04036278e+02],
       [1.22005417e+02, 1.16565063e+02, 1.61565109e+02, ...,
        9.00000000e+01, 9.00000000e+01, 9.00000000e+01],
       [1.10556068e+02, 1.02094765e+02, 1.26869965e+02, ...,
        9.00000000e+01, 9.00000000e+01, 9.00000000e+01]], dtype=float32)

In [20]:
cell_size = 8
block_size = 2
bins = 9
gray = np.array(gray)
img = gray.astype(np.uint8)
h, w = gray.shape
xkernel = np.array([[-1, 0, 1]])
ykernel = np.array([[-1], [0], [1]])
dx = cv2.filter2D(img, cv2.CV_32F, xkernel, borderType = cv2.BORDER_REPLICATE)
print(check(dx, gradient_x))
dy = cv2.filter2D(img, cv2.CV_32F, ykernel, borderType = cv2.BORDER_REPLICATE)
print(check(dy, gradient_y))
magnitude = np.sqrt(np.square(dx) + np.square(dy))
print(check(magnitude, my_magnitude))
orientation = np.arctan(np.divide(dy, dx+0.00001)) # radian
orientation = np.degrees(orientation) # -90 -> 90
orientation += 90 # 0 -> 180
print(check(orientation, my_direction))

num_cell_x = w // cell_size # 8
num_cell_y = h // cell_size # 16
hist_tensor = np.zeros([num_cell_y, num_cell_x, bins]) # 16 x 8 x 9
for cx in range(num_cell_x):
    for cy in range(num_cell_y):
        ori = orientation[cy*cell_size:cy*cell_size+cell_size, cx*cell_size:cx*cell_size+cell_size]
        mag = magnitude[cy*cell_size:cy*cell_size+cell_size, cx*cell_size:cx*cell_size+cell_size]
        hist, _ = np.histogram(ori, bins=bins, range=(0, 180), weights=mag) # 1-D vector, 9 elements
        hist_tensor[cy, cx, :] = hist
    pass
pass
    # normalization
redundant_cell = block_size-1
feature_tensor = np.zeros([num_cell_y-redundant_cell, num_cell_x-redundant_cell, block_size*block_size*bins])
for bx in range(num_cell_x-redundant_cell): # 7
    for by in range(num_cell_y-redundant_cell): # 15
        by_from = by
        by_to = by+block_size
        bx_from = bx
        bx_to = bx+block_size
        v = hist_tensor[by_from:by_to, bx_from:bx_to, :].flatten() # to 1-D array (vector)
        feature_tensor[by, bx, :] = v / LA.norm(v, 2)
        # avoid NaN:
        if np.isnan(feature_tensor[by, bx, :]).any(): # avoid NaN (zero division)
            feature_tensor[by, bx, :] = v
    
f = feature_tensor.flatten() # 3780 features
print('Feature size:', f.shape)
print('Features (HOG):', f)

0.0
0.0
0.0
267346259.72069168


C:\Users\lminh\AppData\Local\Temp\ipykernel_14876\1287362113.py:44: RuntimeWarning: invalid value encountered in divide
  feature_tensor[by, bx, :] = v / LA.norm(v, 2)


Feature size: (2038176,)
Features (HOG): [0.36273813 0.         0.         ... 0.         0.         0.        ]


In [22]:
type(gradient_x)

list